# Lab 3_Supervised Learning

## Prerequisite 
linear regression

target    = cases_recoverd

feature   = date, vaccination, case_new, case_active, case_unvax, case_fvax

KNN regression

target    = cases_recovered

feature   = date, vaccination, case_new, case_active, case_unvax, case_fvax

In [23]:
# Read csv from URL 
import pandas as pd

# Set display options to ensure all data is printed
pd.set_option('display.max_rows', None)
url = "https://raw.githubusercontent.com/MoH-Malaysia/covid19-public/main/epidemic/cases_malaysia.csv"

# Read data from second URL
url2 = "https://raw.githubusercontent.com/CITF-Malaysia/citf-public/main/vaccination/vax_malaysia.csv"


# Filter out data of cases_new, case active, case_unvax, case_fvax, case_recovered according to date (From 1st Sept 2021 to 30th Mac 2021) 
df = pd.read_csv(url)
df['date'] = pd.to_datetime(df['date'])
df = df[(df['date'] >= '2021-09-01') & (df['date'] <= '2022-03-31')]
df = df[['date', 'cases_new', 'cases_active', 'cases_unvax', 'cases_fvax', 'cases_recovered']]

# Filter out data of daily from URL2 according to date (From 1st Sept 2021 to 30th Mac 2021)
df2 = pd.read_csv(url2)
df2['date'] = pd.to_datetime(df2['date'])
df2 = df2[(df2['date'] >= '2021-09-01') & (df2['date'] <= '2022-03-31')]
df2 = df2['daily']

# Combine df2 and df
df = pd.concat([df, df2], axis=1)
df.columns = ['date', 'daily', 'cases_new', 'cases_active', 'cases_unvax', 'cases_fvax', 'cases_recovered']
print(df)   


          date    daily  cases_new  cases_active  cases_unvax  cases_fvax  \
585 2021-09-01  18762.0   265279.0        8299.0       5487.0     21073.0   
586 2021-09-02  20988.0   262545.0        9557.0       6263.0     23473.0   
587 2021-09-03  19378.0   259194.0        8598.0       6015.0     22399.0   
588 2021-09-04  19057.0   256307.0        8308.0       6213.0     21582.0   
589 2021-09-05  20396.0   255794.0        8852.0       6875.0     20573.0   
590 2021-09-06  17352.0   252673.0        7306.0       5969.0     20201.0   
591 2021-09-07  18547.0   252007.0        8142.0       6134.0     18902.0   
592 2021-09-08  19733.0   248678.0        8191.0       7017.0     22701.0   
593 2021-09-09  19307.0   242808.0        8020.0       6930.0     24855.0   
594 2021-09-10  21176.0   242167.0        8437.0       7964.0     21476.0   
595 2021-09-11  19550.0   239354.0        8187.0       7299.0     21771.0   
596 2021-09-12  19198.0   237280.0        7477.0       7871.0     20980.0   